<a href="https://colab.research.google.com/github/padolekrushna/Personalized-Mental-Health-Assistant-/blob/main/Personalized_Mental_Health_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# -*- coding: utf-8 -*-
"""
Personalized Mental Health Assistant

This script builds an advanced mental health assistant that:
1. Takes user queries as input
2. Predicts multiple mental health parameters (emotion, severity, urgency)
3. Generates customized responses using NLP/LLM
4. Suggests appropriate actions based on the analysis
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import json
import warnings
warnings.filterwarnings('ignore')

# Download required NLTK resources
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)

# ===============================
# Step 1: Generate Dataset
# ===============================

def generate_mental_health_dataset(num_samples=1000):
    """
    Generate a synthetic dataset for mental health queries and classifications
    """
    np.random.seed(42)

    # Define possible values for our parameters
    emotions = ['Happy', 'Sad', 'Anxious', 'Angry', 'Overwhelmed', 'Depressed', 'Neutral', 'Confused']
    severity_levels = ['Low', 'Medium', 'High']
    urgency_levels = ['Low', 'Medium', 'High']

    # Templates for generating user queries based on emotions
    query_templates = {
        'Happy': [
            "I'm feeling really good today!",
            "Everything is going so well in my life right now.",
            "I just got some great news and I'm so excited!",
            "I feel more positive than I have in months.",
            "Today was a wonderful day, I accomplished so much."
        ],
        'Sad': [
            "I've been feeling down lately and I don't know why.",
            "I can't seem to enjoy things that used to make me happy.",
            "I've been crying a lot recently for no apparent reason.",
            "I feel so empty inside, like nothing matters anymore.",
            "Everything just feels heavy and pointless lately."
        ],
        'Anxious': [
            "I can't stop worrying about everything that could go wrong.",
            "My heart is racing and I feel like I can't breathe sometimes.",
            "I'm constantly on edge waiting for something bad to happen.",
            "I'm overthinking every little decision and it's exhausting.",
            "My mind won't stop racing with worst-case scenarios."
        ],
        'Angry': [
            "I'm so frustrated with everyone around me right now.",
            "I keep snapping at people for small things and I can't control it.",
            "Everything is making me irritated and I don't know why.",
            "I feel like I'm about to explode with rage over minor issues.",
            "I'm having trouble controlling my temper lately."
        ],
        'Overwhelmed': [
            "There's too much on my plate and I can't handle it all.",
            "I feel like I'm drowning in responsibilities and deadlines.",
            "Everything is piling up and I don't know where to start.",
            "The pressure is just too much for me to handle right now.",
            "I can't keep up with all the demands being placed on me."
        ],
        'Depressed': [
            "I haven't been able to get out of bed for days.",
            "I don't see any point in trying anymore, nothing will change.",
            "I feel completely worthless and like a burden to everyone.",
            "The future looks completely hopeless to me.",
            "I've lost interest in everything I used to care about."
        ],
        'Neutral': [
            "I'm just checking in to talk about my mental health.",
            "Nothing specific is bothering me, just wanted to chat.",
            "I'm feeling okay today, neither good nor bad.",
            "Just wanted to have a conversation about my feelings.",
            "I'm in a pretty average mood, nothing special to report."
        ],
        'Confused': [
            "I don't understand what I'm feeling right now.",
            "My emotions are all over the place and I can't make sense of them.",
            "I feel lost and uncertain about everything.",
            "I can't tell if what I'm experiencing is normal or not.",
            "My thoughts are jumbled and I can't sort them out."
        ]
    }

    # Rule-based mapping of emotion to likely severity and urgency
    severity_mapping = {
        'Happy': ['Low'] * 90 + ['Medium'] * 10,
        'Sad': ['Low'] * 30 + ['Medium'] * 60 + ['High'] * 10,
        'Anxious': ['Low'] * 20 + ['Medium'] * 60 + ['High'] * 20,
        'Angry': ['Low'] * 30 + ['Medium'] * 50 + ['High'] * 20,
        'Overwhelmed': ['Medium'] * 70 + ['High'] * 30,
        'Depressed': ['Medium'] * 30 + ['High'] * 70,
        'Neutral': ['Low'] * 80 + ['Medium'] * 20,
        'Confused': ['Low'] * 40 + ['Medium'] * 50 + ['High'] * 10
    }

    urgency_mapping = {
        'Happy': ['Low'] * 95 + ['Medium'] * 5,
        'Sad': ['Low'] * 40 + ['Medium'] * 50 + ['High'] * 10,
        'Anxious': ['Low'] * 30 + ['Medium'] * 50 + ['High'] * 20,
        'Angry': ['Low'] * 40 + ['Medium'] * 40 + ['High'] * 20,
        'Overwhelmed': ['Medium'] * 60 + ['High'] * 40,
        'Depressed': ['Medium'] * 20 + ['High'] * 80,
        'Neutral': ['Low'] * 90 + ['Medium'] * 10,
        'Confused': ['Low'] * 50 + ['Medium'] * 40 + ['High'] * 10
    }

    # Action suggestions based on the combined parameters
    action_suggestions = {
        ('Happy', 'Low', 'Low'): 'Continue with your current positive activities',
        ('Happy', 'Medium', 'Low'): 'Share your positive experiences with others',
        ('Sad', 'Low', 'Low'): 'Try engaging in activities you usually enjoy',
        ('Sad', 'Medium', 'Medium'): 'Consider talking to a friend or family member about your feelings',
        ('Sad', 'High', 'High'): 'Please consider scheduling an appointment with a mental health professional',
        ('Anxious', 'Low', 'Low'): 'Practice deep breathing exercises when you feel anxious',
        ('Anxious', 'Medium', 'Medium'): 'Try mindfulness meditation to manage your anxiety',
        ('Anxious', 'High', 'High'): 'Consider consulting with a therapist who specializes in anxiety',
        ('Angry', 'Low', 'Low'): 'Take a short break from the situation to cool down',
        ('Angry', 'Medium', 'Medium'): 'Try physical exercise to release tension',
        ('Angry', 'High', 'High'): 'Consider anger management techniques or professional support',
        ('Overwhelmed', 'Medium', 'Medium'): 'Break down your tasks into smaller, manageable steps',
        ('Overwhelmed', 'High', 'High'): 'Consider delegating tasks and seeking support from others',
        ('Depressed', 'Medium', 'Medium'): 'Establish a regular routine and try to get adequate sleep',
        ('Depressed', 'High', 'High'): 'Please reach out to a mental health professional as soon as possible',
        ('Neutral', 'Low', 'Low'): 'Consider journaling to better understand your emotions',
        ('Confused', 'Low', 'Low'): 'Take some time for self-reflection to identify your feelings',
        ('Confused', 'Medium', 'Medium'): 'Try discussing your thoughts with someone you trust'
    }

    # Default action suggestions for combinations not explicitly defined
    default_action_suggestions = {
        'Low': 'Monitor your feelings and practice self-care',
        'Medium': 'Consider reaching out for support from friends or family',
        'High': 'It might be beneficial to consult with a mental health professional'
    }

    # Generate the dataset
    data = []
    for _ in range(num_samples):
        # Randomly select an emotion
        emotion = np.random.choice(emotions)

        # Select a template for that emotion and modify it slightly
        template = np.random.choice(query_templates[emotion])
        words = template.split()

        # Add some randomness to the query (add/remove words, change order slightly)
        if len(words) > 5 and np.random.random() < 0.3:
            # Remove a random word
            idx_to_remove = np.random.randint(0, len(words))
            words.pop(idx_to_remove)

        if np.random.random() < 0.3:
            # Add a filler word
            filler_words = ['really', 'just', 'kind of', 'sort of', 'a bit', 'somewhat', 'extremely']
            words.insert(np.random.randint(0, len(words)), np.random.choice(filler_words))

        user_query = ' '.join(words)

        # Determine severity and urgency based on emotion
        severity = np.random.choice(severity_mapping[emotion])
        urgency = np.random.choice(urgency_mapping[emotion])

        # Determine action suggestion
        key = (emotion, severity, urgency)
        if key in action_suggestions:
            action = action_suggestions[key]
        else:
            action = default_action_suggestions[severity]

        data.append({
            'User_Query': user_query,
            'Emotion': emotion,
            'Severity': severity,
            'Urgency': urgency,
            'Suggested_Action': action
        })

    return pd.DataFrame(data)

# Generate and save the dataset
df = generate_mental_health_dataset(1500)
df.to_excel("mental_health_assistant_dataset.xlsx", index=False)
print(f"Generated dataset with {len(df)} samples.")

# ===============================
# Step 2: Data Preprocessing
# ===============================

def preprocess_text(text):
    """
    Preprocess text data for NLP
    """
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    # Clean text
    text = re.sub(r'http\S+', '', str(text))  # Remove URLs
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.lower()  # Convert to lowercase
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

# Apply preprocessing to the user queries
df['Processed_Text'] = df['User_Query'].apply(preprocess_text)

# Display dataset information
print("\nDataset Overview:")
print(df.head())
print("\nDataset Information:")
print(df.info())
print("\nClass Distributions:")
for col in ['Emotion', 'Severity', 'Urgency']:
    print(f"\n{col} Distribution:")
    print(df[col].value_counts())

# ===============================
# Step 3: Data Visualization
# ===============================

# Visualize the distribution of emotions
plt.figure(figsize=(12, 6))
sns.countplot(y=df['Emotion'], order=df['Emotion'].value_counts().index)
plt.title('Distribution of Emotions in Dataset')
plt.tight_layout()
plt.savefig('emotion_distribution.png')

# Visualize the relationship between emotion and severity
plt.figure(figsize=(14, 8))
emotion_severity = pd.crosstab(df['Emotion'], df['Severity'])
emotion_severity_pct = emotion_severity.div(emotion_severity.sum(axis=1), axis=0) * 100
emotion_severity_pct.plot(kind='bar', stacked=True, colormap='viridis')
plt.title('Relationship Between Emotion and Severity')
plt.xlabel('Emotion')
plt.ylabel('Percentage')
plt.legend(title='Severity')
plt.tight_layout()
plt.savefig('emotion_severity_relationship.png')

# ===============================
# Step 4: Feature Engineering and Model Building
# ===============================

# Encode target labels
label_encoders = {}
for column in ['Emotion', 'Severity', 'Urgency']:
    le = LabelEncoder()
    df[f'{column}_Encoded'] = le.fit_transform(df[column])
    label_encoders[column] = le

# Prepare features and multiple targets
X = df['Processed_Text']
y = df[['Emotion_Encoded', 'Severity_Encoded', 'Urgency_Encoded']]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with TF-IDF and multi-output classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),
    ('classifier', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42)))
])

# Train the model
print("\nTraining the multi-output model...")
pipeline.fit(X_train, y_train)

# Evaluate the model
print("\nEvaluating the model...")
y_pred = pipeline.predict(X_test)

# Output evaluation metrics for each parameter
target_names = ['Emotion', 'Severity', 'Urgency']
for i, name in enumerate(target_names):
    print(f"\n{name} Classification Report:")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i],
                               target_names=label_encoders[name].classes_))
    print(f"{name} Accuracy:", accuracy_score(y_test.iloc[:, i], y_pred[:, i]))

    # Create confusion matrix
    cm = confusion_matrix(y_test.iloc[:, i], y_pred[:, i])
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
               xticklabels=label_encoders[name].classes_,
               yticklabels=label_encoders[name].classes_)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix for {name}')
    plt.tight_layout()
    plt.savefig(f'{name.lower()}_confusion_matrix.png')

# ===============================
# Step 5: Build Response Generator
# ===============================

def generate_llm_response(emotion, severity, urgency, query):
    """
    Generate a customized response based on the predicted parameters
    This is a rule-based implementation, but could be replaced with an actual LLM API call
    """
    # Base responses for each emotion
    emotion_responses = {
        'Happy': [
            "It's wonderful to hear you're feeling positive! This kind of emotional state is great for building resilience.",
            "I'm glad you're in good spirits! These moments of happiness can be really valuable.",
            "It's great that you're feeling happy. These positive emotions help strengthen our mental wellbeing."
        ],
        'Sad': [
            "I'm sorry to hear you're feeling down. It's completely normal to experience sadness at times.",
            "I can sense you're feeling sad. Remember that it's okay to not be okay sometimes.",
            "I understand you're feeling sad right now. These emotions, while difficult, are a natural part of life."
        ],
        'Anxious': [
            "I notice you're feeling anxious. Many people experience anxiety, and there are effective ways to manage it.",
            "It sounds like you're experiencing some anxiety. This is a common response to stress.",
            "I can tell you're feeling anxious. Your brain is trying to protect you, but sometimes it overestimates threats."
        ],
        'Angry': [
            "I can see you're feeling frustrated. Anger is often a signal that something important to you has been violated.",
            "It sounds like you're feeling angry, which is a natural response to perceived unfairness or obstacles.",
            "I understand you're feeling angry. This emotion can actually be informative about your boundaries and values."
        ],
        'Overwhelmed': [
            "It seems like you're feeling overwhelmed. When everything feels too much, breaking things down can help.",
            "I can tell you're overwhelmed right now. It's a common feeling when we're faced with multiple challenges.",
            "I understand that feeling of being overwhelmed. Your mind is trying to process a lot at once."
        ],
        'Depressed': [
            "I'm hearing that you may be experiencing depression. This is a real condition that can affect anyone.",
            "It sounds like you might be dealing with depression. These feelings are valid and deserve attention.",
            "I understand you're feeling symptoms of depression. This isn't something you have to face alone."
        ],
        'Neutral': [
            "Thank you for checking in. Regular emotional check-ins are a good practice for mental health.",
            "I appreciate you sharing where you're at emotionally. Even neutral states are worth noting.",
            "Thank you for the update. Being aware of our emotional state, even when neutral, is valuable."
        ],
        'Confused': [
            "It sounds like you're feeling uncertain about your emotions. This kind of emotional confusion is actually quite common.",
            "I can understand that feeling of confusion. Sometimes our emotions aren't straightforward.",
            "I hear that you're having trouble pinpointing exactly what you're feeling. That's perfectly normal."
        ]
    }

    # Severity modifiers
    severity_modifiers = {
        'Low': [
            "This seems to be a mild experience for you.",
            "From what you've shared, this isn't severely impacting your daily life.",
            "It sounds like you're managing this well overall."
        ],
        'Medium': [
            "This seems to be having a noticeable impact on you.",
            "I can tell this is affecting your wellbeing to some degree.",
            "This sounds like it's creating some significant challenges for you."
        ],
        'High': [
            "This appears to be having a substantial impact on your wellbeing.",
            "I can see this is significantly affecting your daily functioning.",
            "This sounds like it's creating major challenges for you right now."
        ]
    }

    # Urgency response components
    urgency_responses = {
        'Low': [
            "There's no rush to address this immediately, but it's good to be aware of.",
            "This isn't an emergency situation, but it's still worth your attention.",
            "While not urgent, this is still important to acknowledge."
        ],
        'Medium': [
            "It would be beneficial to address this relatively soon.",
            "This deserves your attention in the near future.",
            "Consider making this a priority in your self-care."
        ],
        'High': [
            "I recommend addressing this as soon as possible.",
            "This situation deserves immediate attention.",
            "It's important that you take care of this right away."
        ]
    }

    # Select random components for variety
    emotion_response = np.random.choice(emotion_responses[emotion])
    severity_modifier = np.random.choice(severity_modifiers[severity])
    urgency_response = np.random.choice(urgency_responses[urgency])

    # Construct the complete response
    response = f"{emotion_response} {severity_modifier} {urgency_response}"

    return response

# ===============================
# Step 6: Action Suggestion Generator
# ===============================

def get_action_suggestion(emotion, severity, urgency):
    """
    Generate action suggestions based on predicted parameters
    """
    # Comprehensive mapping of parameter combinations to suggestions
    action_mapping = {
        # Happy
        ('Happy', 'Low', 'Low'): [
            "Continue engaging in activities that bring you joy.",
            "Consider sharing your positive experiences with others.",
            "Take note of what's contributing to your happiness for future reference."
        ],
        ('Happy', 'Medium', 'Low'): [
            "Channel this positive energy into creative projects or goals.",
            "Use this positive state to connect with others.",
            "Consider journaling about what's going well to revisit during harder times."
        ],

        # Sad
        ('Sad', 'Low', 'Low'): [
            "Try engaging in a small activity you usually enjoy.",
            "Consider light exercise or time in nature.",
            "Practice self-compassion and remember that all emotions are temporary."
        ],
        ('Sad', 'Medium', 'Medium'): [
            "Reach out to a trusted friend or family member.",
            "Consider scheduling a pleasant activity for yourself.",
            "Try mindfulness meditation to acknowledge your feelings without judgment."
        ],
        ('Sad', 'High', 'High'): [
            "Please consider speaking with a mental health professional.",
            "Reach out to a crisis support line if you need immediate support.",
            "Don't hesitate to talk to your doctor about how you're feeling."
        ],

        # Anxious
        ('Anxious', 'Low', 'Low'): [
            "Practice deep breathing exercises when you notice anxiety.",
            "Try grounding techniques like the 5-4-3-2-1 method.",
            "Consider limiting caffeine and getting regular physical activity."
        ],
        ('Anxious', 'Medium', 'Medium'): [
            "Schedule dedicated 'worry time' to contain anxious thoughts.",
            "Try progressive muscle relaxation techniques.",
            "Consider using a meditation app designed for anxiety."
        ],
        ('Anxious', 'High', 'High'): [
            "Please consider consulting with a mental health professional who specializes in anxiety.",
            "Look into cognitive-behavioral therapy techniques for anxiety.",
            "Speak with your doctor about treatment options if anxiety is significantly impacting your life."
        ],

        # Overwhelming
        ('Overwhelmed', 'Medium', 'Medium'): [
            "Break down what's overwhelming you into smaller, manageable tasks.",
            "Identify what you can control and what you can't, focusing on the former.",
            "Consider using organizational tools to help manage your responsibilities."
        ],
        ('Overwhelmed', 'High', 'High'): [
            "Prioritize self-care and consider what responsibilities can be delegated or postponed.",
            "Reach out for support from friends, family, or colleagues.",
            "Consider speaking with a therapist about developing coping strategies."
        ]
    }

    # Default suggestions based on severity when specific combination isn't mapped
    default_suggestions = {
        'Low': [
            "Continue monitoring your feelings and practice regular self-care.",
            "Consider keeping a mood journal to track patterns in your emotions.",
            "Maintain healthy habits like regular sleep, nutrition, and exercise."
        ],
        'Medium': [
            "Reach out to your support network to discuss what you're experiencing.",
            "Consider implementing structured self-care activities into your routine.",
            "Look into resources like books or podcasts about emotional wellbeing."
        ],
        'High': [
            "It would be beneficial to consult with a mental health professional.",
            "Consider whether your symptoms warrant a conversation with your doctor.",
            "Look into local mental health resources or support groups."
        ]
    }

    # Get appropriate suggestions
    key = (emotion, severity, urgency)
    if key in action_mapping:
        suggestions = action_mapping[key]
    else:
        suggestions = default_suggestions[severity]

    # Return a random suggestion for variety
    return np.random.choice(suggestions)

# ===============================
# Step 7: Create the User Interface
# ===============================

def process_user_query(query):
    """
    Process a user query and return the analysis results
    """
    # Preprocess the query
    processed_query = preprocess_text(query)

    # Make predictions
    predictions = pipeline.predict([processed_query])[0]

    # Decode predictions
    emotion = label_encoders['Emotion'].inverse_transform([predictions[0]])[0]
    severity = label_encoders['Severity'].inverse_transform([predictions[1]])[0]
    urgency = label_encoders['Urgency'].inverse_transform([predictions[2]])[0]

    # Generate response and action suggestions
    response = generate_llm_response(emotion, severity, urgency, query)
    action = get_action_suggestion(emotion, severity, urgency)

    # Return the results
    results = {
        'Query': query,
        'Analysis': {
            'Emotion': emotion,
            'Severity': severity,
            'Urgency': urgency
        },
        'Response': response,
        'Suggested_Action': action
    }

    return results

# ===============================
# Step 8: Demo and Testing
# ===============================

def run_demo():
    """
    Run a demo of the mental health assistant
    """
    print("\n" + "="*50)
    print("Personalized Mental Health Assistant Demo")
    print("="*50)

    test_queries = [
        "I feel like I'm drowning in work and can't take it anymore.",
        "I've been feeling really happy and energetic lately.",
        "I can't stop worrying about my future and it's keeping me up at night.",
        "I just feel empty inside, like nothing matters.",
        "I get so angry at the smallest things and I don't know why."
    ]

    for query in test_queries:
        print("\nUser Query:", query)
        results = process_user_query(query)

        print("\nAnalysis:")
        for param, value in results['Analysis'].items():
            print(f"- {param}: {value}")

        print("\nResponse:")
        print(results['Response'])

        print("\nSuggested Action:")
        print(results['Suggested_Action'])

        print("\n" + "-"*50)

    # Interactive mode
    print("\nInteractive Mode (type 'exit' to quit):")
    while True:
        user_input = input("\nEnter your query: ")
        if user_input.lower() == 'exit':
            break

        results = process_user_query(user_input)

        print("\nAnalysis:")
        for param, value in results['Analysis'].items():
            print(f"- {param}: {value}")

        print("\nResponse:")
        print(results['Response'])

        print("\nSuggested Action:")
        print(results['Suggested_Action'])

# ===============================
# Step 9: Save the Model and Run
# ===============================

import joblib

# Save the model and encoders
print("\nSaving the model and resources...")
joblib.dump(pipeline, 'mental_health_model.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')

print("\nModel and resources saved successfully!")

# Run the demo
if __name__ == "__main__":
    run_demo()

Generated dataset with 1500 samples.

Dataset Overview:
                                          User_Query    Emotion Severity  \
0  Just wanted to have a conversation about my fe...    Neutral      Low   
1  Nothing specific is bothering me, just wanted ...    Neutral      Low   
2  I've lost interest in everything I used to car...  Depressed     High   
3  I can't tell if what I'm experiencing is norma...   Confused   Medium   
4  I'm constantly sort of on edge waiting for som...    Anxious   Medium   

  Urgency                                   Suggested_Action  \
0     Low  Consider journaling to better understand your ...   
1     Low  Consider journaling to better understand your ...   
2  Medium  It might be beneficial to consult with a menta...   
3     Low  Consider reaching out for support from friends...   
4  Medium  Try mindfulness meditation to manage your anxiety   

                                      Processed_Text  
0                        wanted conversation fe